In [225]:
#### Libs ###
from typing_extensions import TypedDict
from typing import Annotated, Dict, Any
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langchain_core.messages import AIMessage
import requests
import time
import pyodbc
from contextlib import closing
import pandas as pd
from warnings import filterwarnings
import requests
filterwarnings('ignore')

In [226]:
class config:
    conn_str = (
        "DRIVER={SQL Server};"
        "SERVER=216.48.191.98;"
        "DATABASE=Agentic_Automation;"
        "UID=ibsadmin;"
        "PWD=Viking@@ibs2023;"
    )
conn=pyodbc.connect(config.conn_str)
cur=conn.cursor()

In [227]:
# iterate task processing by order task_insert_time, task_number where task_Status ='New'
# check previous task of same webtop is completed if yes than run or pick next record

In [228]:

query_pick_records='''
select tm.CloudFlowApi,tp.Webtop_ID,tm.Process_Name,tp.Work_ID,tm.Task_Name,tp.Task_Number from Credit_GPT.dbo.Task_Processing as tp 
--left join Credit_GPT.dbo.[Task_Dependencies] as td on tp.Task_Number=td.Task_Number 
left join Credit_GPT.dbo.Task_Master as tm on tp.Task_Number=tm.Task_Number
where tp.Task_Status='New' --or tp.Task_Status'In Progress'
order by tp.task_insert_time,CAST(tp.Task_Number AS BIGINT)
'''
def check_dependency_record_status(task_number,webtop_id):
    query_dependency_record=f'''
select Task_Number,Task_Name,Task_Status,Failure_Reason from Credit_GPT.dbo.Task_Processing where Task_Number in 
(select Dependency_Task_Number from Credit_GPT.dbo.Task_Dependencies where Webtop_ID='{webtop_id}' and Task_Number='{task_number}') 
'''
    df_dependency_records=pd.read_sql(query_dependency_record,conn)
    
    if len(df_dependency_records)==0:
        dependency_record_Status=("No Dependency tasks is there",df_dependency_records)
        return dependency_record_Status

    df_dependency_completed_records=df_dependency_records[df_dependency_records['Task_Status']=="Completed"]

    if len(df_dependency_completed_records)==len(df_dependency_records):
        dependency_record_Status=("Dependency tasks Are Completed",df_dependency_records)
    else:
        dependency_record_Status=("Dependency tasks Are Not Completed",df_dependency_records)
    return dependency_record_Status


In [229]:
# check_previous_record_status('3','ABC')
df_pick_records=pd.read_sql(query_pick_records,conn)
df_pick_records

,CloudFlowApi,Webtop_ID,Process_Name,Work_ID,Task_Name,Task_Number
0,https://prod-01.centralindia.logic.azure.com:4...,349PZ8633504,Bureau Deviation,work1,ReportCreation,11


In [230]:
# url=df_pick_records.loc[0,'CloudFlowApi']
# print(url)
# body={
#         "Webtop_ID": "349PZ8633504",
#         "Process_Name": "Bureau Analysis",
#         "Work_ID": "work1",
#         "Task_Name":"DocDownload"
# }
# response=requests.post(url=url,json=body)
# response.raise_for_status()
# print(response)
def run_flow(url,webtop,process_name,work_id,task_name):
    body={
            "Webtop_ID":webtop,
            "Process_Name":process_name,
            "Work_ID":work_id,
            "Task_Name":task_name
            }
    response=requests.post(url,json=body)
    response.raise_for_status()
    return response

In [231]:
while True:
    df_pick_records=pd.read_sql(query_pick_records,conn)
    for _,row in df_pick_records.iterrows():
        CloudFlowApi=row['CloudFlowApi']
        Webtop_ID=row['Webtop_ID']
        Process_Name=row['Process_Name']
        Work_ID=row['Work_ID']
        Task_Name=row['Task_Name']
        Task_Number=row['Task_Number']
        print(Task_Name,end=" ")
        previous_record_status,df=check_dependency_record_status(Task_Number,Webtop_ID)
        if previous_record_status=='Dependency tasks Are Completed' or previous_record_status=="No Dependency tasks is there":
            print(previous_record_status)
            print(f"==>{Task_Name} is Started for {Webtop_ID}, for Process {Process_Name} and Work_ID is {Work_ID}")
            response=run_flow(CloudFlowApi,Webtop_ID,Process_Name,Work_ID,Task_Name)
            cur.execute(f"update Credit_GPT.dbo.Task_Processing set Task_Status='In Progress' where Task_Number='{Task_Number}' and Webtop_ID='{Webtop_ID}'")
            conn.commit()
        else:
            print(previous_record_status)
            print(df)
            time.sleep(5)

            continue
        time.sleep(5)




        
    

ReportCreation No Dependency tasks is there
==>ReportCreation is Started for 349PZ8633504, for Process Bureau Deviation and Work_ID is work1
document_download No Dependency tasks is there
==>document_download is Started for qwertyuiop1234567890, for Process Credit_Ops and Work_ID is b9ead074-f041-444a-a362-890f60b0110c
crm_scraping No Dependency tasks is there
==>crm_scraping is Started for qwertyuiop1234567890, for Process Credit_Ops and Work_ID is b9ead074-f041-444a-a362-890f60b0110c
document_download No Dependency tasks is there
==>document_download is Started for qwertyuiop1234567890, for Process Credit_Ops and Work_ID is b9ead074-f041-444a-a362-890f60b0110c
data_extraction Dependency tasks Are Not Completed
  Task_Number          Task_Name  Task_Status Failure_Reason
0           2  document_download  In Progress           None
data_validation Dependency tasks Are Not Completed
  Task_Number        Task_Name  Task_Status Failure_Reason
0           1     crm_scraping  In Progress   

MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant https://?